In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
import numpy.random as r
from tqdm import tqdm

# Her i starten indlæses bare data og korrigeres for k_acc, som vi har gjort tidligere. Led efter næste markdown, hvor der står, at der er ændringer

In [2]:
rådata_holocæn = pd.read_excel('Temperaturdata_alene.xlsx', header = 29)
rådata_holocæn = rådata_holocæn.to_numpy()
år20_rå = rådata_holocæn[:,0]
temp20_rå = rådata_holocæn[:,1]

#plt.plot(år20_rå,temp20_rå)
#
#plt.xlabel('Tid tilbage [år]', fontsize = 13)
#plt.ylabel('Temperaturafigelse [grader celsius]', fontsize = 13)
#plt.title("Gennemsnitlig temperaturafvigelse gennem den Holocæne periode", fontsize = 13)

In [3]:
rådata = pd.read_excel('Ægte_DailyMinMax_Ilullisat_Cor-2.xlsx')
#print(rådata)
data = rådata.to_numpy()
år_rå, temp_rå = data[:,0], data[:,1]

Interpolering af DMI-data

In [4]:
def interpolation_år():
    interpol_år = []
    år = []
    temp = []
    for i in range(1, len(år_rå)):      # Finder mellempunkter for årsarrayet
        punkt = (år_rå[i] + år_rå[i-1])/2
        interpol_år.append(punkt)
    
    for i in range(len(interpol_år)):   # Laver liste med både DMI-årstal og mellempunkter
        år.append(år_rå[i])
        år.append(interpol_år[i])
    #print(interpol_år)
    år.append(år_rå[-1])   # Tilføjer det sidste rå datapunkt manuelt, da der et ét råt datapunkt mere end mellempunkterne.
    print(len(år))
    return år
    
år_DMI_inter = interpolation_år()      # Interpolerer temperaturdataen

år_DMI_inter = np.array(år_DMI_inter)

temp_DMI_inter = np.interp(år_DMI_inter, år_rå, temp_rå)

#plt.plot(år_DMI_inter, temp_DMI_inter)
#
#print(len(år_DMI_inter), len(temp_DMI_inter))

216227


In [5]:
råsatellit = pd.read_csv('greenland_mass_200204_202311.txt', delimiter = '\s+', header = None, skiprows = lambda x: x < 31)
satellit = råsatellit.to_numpy()
år_sat, dmasse_sat = satellit[:, 0], satellit[:, 1]
densitet_is = 917
dvol_sat = 10**12 * dmasse_sat/densitet_is   # Vi omregner massetilvækst til volumentilvækst
V_sat_err = 10**12 * abs(1/densitet_is * satellit[:,2])

In [6]:
top = max(år_sat)   
bund = min(år_sat)

# Slicing af DMI-data

temp_DMI = temp_DMI_inter[(år_DMI_inter <= top) & (år_DMI_inter >= bund)] # Nyt temperatur-array, som kun indeholder temperaturer for samme årrække som satellitdata
år_DMI = år_DMI_inter[(år_DMI_inter <= top) & (år_DMI_inter >= bund)] # Sørger for, at vi også har et årstals-array med de tilsvarende indgange (samme maske)

# Slicing af GRACE-data
topDMI = max(år_DMI)
dmasse_sat = dmasse_sat[år_sat <= topDMI]
dvol_sat = dvol_sat[år_sat <= topDMI]
V_sat_err = V_sat_err[år_sat <= topDMI]
år_sat = år_sat[år_sat <= topDMI]

In [7]:
råNGRIP = pd.read_excel('rådata NGRIP_acc_Vinther_etal_2009_GICC05_16Sep2010.xlsx', header = 58, usecols = [0,1,2,3,4,5,6,7,8])
råGRIP = pd.read_excel('GRIP_acc_Vinther_etal_2009_GICC05_16Sep2010.xlsx', header = 54, usecols = [0,1,2,3,4,5,6,7])

NGRIP, GRIP = råNGRIP.to_numpy(), råGRIP.to_numpy()
NGRIPakk0, GRIPakk0 = NGRIP[:, 6], GRIP[:, 5]
NGRIPår0, GRIPår0 = NGRIP[:, 1], GRIP[:, 1]
NGRIP_delevation, GRIP_delevation = NGRIP[:, 8], GRIP[:, 7]

In [8]:
akk_list = (NGRIPakk0[1:21] + GRIPakk0[1:21])/2   # Tager ikke første punkt med grunden manglende nedbørsdata
akk_gns = np.mean(akk_list)
print('gns akkumulation = ', akk_gns)

gns akkumulation =  0.2034321448890318


In [9]:
def optimering_akk(M_akk_tabel):

    dvol_sat_korr = dvol_sat - np.mean(dvol_sat) # volumenændringer for GRACE korrigeres, så det har nulpunkt i sit gennemsnit


    V_start = 2.99 * 10**15   # startvolumen af iskappe
    H = 3 * 10**3       # Starthøjde af iskappe
    V = V_start  # startvolumen af iskappe
    V_0 = V_start 
    V_sat = dvol_sat + V_0 - dvol_sat[0]
    
    R = (3/16*V/H)**(1/2)    # Startradius af iskappe
    T_f = 0        
    alfa = - 9.8 * 10**(-3)  # konstant hvormed temperatur stiger per meter
    k = H/np.sqrt(R)   # formfaktor for iskappe. Antages global konstant
    dt = 1/(365.25 * 4)  # Tidsskridt (her 1/4 døgn) - omregnet til år

    acc = akk_gns   # Nedbør pr år [m/år]. Antages konstant i perioden
    k_acc_test = 1   # Denne tuningskonstant kommer fra vores undervurdering af nedbøren. Den kan vi også tune på, hvis nødvendigt. 
    k_melt = 1

    #Der oprettes tomme lister til resultatet af simuleringen
    V_list = []
    V_melt_list = []
    V_acc_list = []


    for i in range(len(år_DMI)):
        R = (3/16 * V/k)**(2/5)     # Ny radius af iskappe
       
        V_acc = k_acc_test * (acc * 8 * R**2) * dt  # Nedbøret dette tidsskridt
       
        if temp_DMI[i] - T_f <= 0:
            V_melt = 0
        elif temp_DMI[i] - T_f > 0:
            V_melt = k_melt * 8/3 * R * (temp_DMI[i] - T_f) * ((T_f - temp_DMI[i])/(alfa * k))**2 * dt  # ganger tidsskridt og M på
        V += V_acc - V_melt

        V_list.append(V)
        V_melt_list.append(V_melt)
        V_acc_list.append(V_acc)

   
    V_sum = np.sum(V_acc_list)/(år_DMI[-1] - år_DMI[0])
    V_akk_tabel = (M_akk_tabel)/(densitet_is) #m^3

    k_acc = V_akk_tabel/V_sum
    
    return k_acc

In [10]:
M_akk_mean = 642 * 10**12    #kg
M_akk_std = 59 * 10**12    #kg

M_akk_list = r.normal(M_akk_mean, M_akk_std, 100)

k_acc_værdier = []

for x in M_akk_list:
    k_acc = optimering_akk(x)

    k_acc_værdier.append(k_acc)

#print(k_acc_værdier)

# __HERFRA NYT__

In [11]:
V_start = 2.99 * 10**15   # startvolumen af iskappe
dvol_sat_korr = dvol_sat - np.mean(dvol_sat) # volumenændringer for GRACE korrigeres, så det har nulpunkt i sit gennemsnit
dvol_sat_err_korr = np.sqrt(V_sat_err**2 + (np.std(dvol_sat)/np.sqrt(len(dvol_sat)))**2)


def simulering(år_sat, k_melt):
    
    # De globale konstanter defineres
    V = V_start  # startvolumen af iskappe
    V_0 = V_start 
    H = 3 * 10**3       # Starthøjde af iskappe
    R = (3/16*V/H)**(1/2)     # Startradius af iskappe
    T_f = 0        
    alfa = - 9.8 * 10**(-3)  # konstant hvormed temperatur stiger per meter
    k = H/np.sqrt(R)   # formfaktor for iskappe. Antages global konstant
    dt = 1/(365.25 * 4)  # Tidsskridt (her 1/4 døgn) - omregnet til år
    
    acc = akk_gns   # Nedbør pr år [m/år]. Antages konstant i perioden

    
    # Der oprettes tomme lister til resultatet af simuleringen
    V_list = []
    V_melt_list = []
    V_acc_list = []
    
    
    for i in range(1, len(år_DMI) + 1):
        R = (3/16 * V/k)**(2/5)     # Ny radius af iskappe
        
        V_acc = k_acc * (acc * 8 * R**2) * dt  # Nedbøret dette tidsskridt
        
        if temp_DMI[-i] - T_f <= 0:
            V_melt = 0
        elif temp_DMI[-i] - T_f > 0:
            V_melt = k_melt * 8/3 * R * (temp_DMI[-i] - T_f) * ((T_f - temp_DMI[-i])/(alfa * k))**2 * dt  # ganger tidsskridt og M på
        V +=  - V_acc + V_melt
        
        V_list.append(V)
        V_melt_list.append(V_melt)
        V_acc_list.append(V_acc)
    
    V_list.reverse()

    dvol_sim = V_list - np.mean(V_list)

    dvol_sim_fit = np.interp(år_sat, år_DMI, dvol_sim)

    return dvol_sim_fit 


# Nedenstående er bare et tjek af den nye funktion, som er lidt anderledes end de gamle
#test = simulering(år_sat, 6.7)
#
##plt.plot(år, V_list)
#plt.plot(år_sat, test, label = 'simulering')
#plt.plot(år_sat, dvol_sat_korr)
#plt.legend()


## Curve_fit

In [12]:
k_melt_list = []
k_melt_err_list = []

for x in k_acc_værdier:
    k_acc = x
    par, cov = curve_fit(simulering, år_sat, dvol_sat_korr, p0 = 1, sigma = dvol_sat_err_korr, absolute_sigma= True)
    k_melt_list.append(par[0])
    k_melt_err_list.append(np.sqrt(cov[0,0]))

In [13]:
print(len(k_melt_list))
print(k_melt_list)
print(k_melt_err_list)

100
[7.793764806783167, 8.066439909733624, 8.798250116456186, 8.369568407308597, 7.583058371775736, 9.02966079017822, 8.444690950870488, 8.7360091451117, 8.58085111294115, 6.845915035030717, 7.787370914025731, 8.101440109334638, 8.550648832029488, 7.770889802046215, 8.813043549044316, 7.431996229070866, 8.37082233311917, 8.570231711916907, 8.3934100545947, 7.8508030569805545, 8.20367124406362, 7.112241002945523, 7.899988360904909, 8.298986466274487, 7.660403611035217, 8.614559523240887, 8.741934271495928, 8.210505773136651, 8.665430103697544, 7.26133525160456, 8.592692324250804, 7.894232101503376, 7.787687779435537, 7.468030195535627, 8.528107411308731, 8.304767422554736, 7.432037597358869, 7.673241543151826, 8.425668032921186, 7.871840485998432, 7.294544075088718, 8.887295654589012, 8.421203330642099, 6.923230099609211, 8.882657907624347, 8.998438724706128, 7.435302482416992, 7.386962313894497, 7.502145778958407, 8.26109214115033, 7.261203235236637, 8.002651784059083, 7.61535365054278

# __Lange kørsler__

In [16]:
gns1 = np.average(temp20_rå[0:5])

top = 1980
bund = 1880
temp_X = temp_rå[(år_rå >= bund)&(år_rå < top)]

gns2 = np.average(temp_X)

forskel = np.absolute(gns1 - gns2)

#plt.plot(np.linspace(1880,1980,len(temp_X)),temp_X, '.', c = 'skyblue', label = 'Meteorologisk data')
#plt.plot(np.linspace(1880,1980,100), np.linspace(gns2,gns2,100), c = 'lightseagreen', label = 'Gennemsnit for meteorologisk data')
#
#plt.plot(np.linspace(1880,1980,5),temp20_rå[0:5], '.', c = 'darkorange', label = 'Iskernedatadata')
#plt.plot(np.linspace(1880,1980,100), np.linspace(gns1,gns1,100), c = 'orange', label = 'Gennemsnit for iskernedata')
#
#plt.legend()
#plt.xlabel('Tid [år]', fontsize = 13)
#plt.ylabel(r'Temperatur [$^\circ$C]', fontsize = 13)
#
#print("Forskellen er", forskel)

## Simulering af iskappevolumensudvikling gennem den sidste varmeperiode

### Udvælger 20 år (år 1955-1975) og bruger deres årstidscyklus på iskernedata

In [17]:
top = 1975
bund = 1955
maske = (år_DMI_inter >= bund) & (år_DMI_inter < top)

år_X = år_DMI_inter[maske]
år_X = år_X - år_X[0]
print(år_X)
print(min(år_X))
temp_X = temp_DMI_inter[maske]
temp_X_afv = temp_X-np.average(temp_X)

#print(år_X)
#plt.plot(år_X,temp_X_afv, c = 'darkorange')
#plt.xlabel('Tid [år]', fontsize = 14)
#plt.ylabel(r'Temperaturafvigelse [$^\circ$C]', fontsize = 14)

[0.000000e+00 7.000000e-04 1.400000e-03 ... 1.999795e+01 1.999860e+01
 1.999930e+01]
0.0


## Lægger årstidscyklus ind over iskernedata

In [18]:
temp_array = np.array([])
år_array = np.array([])

for i, j in np.nditer([temp20_rå, år20_rå]):
    temp_c = i + temp_X - forskel #kysttemperatur
    år_c = 2000 - j + 20 + år_X

    temp_array = np.append(temp_array, temp_c)
    år_array = np.append(år_array, år_c)

In [19]:
#plt.plot(år_array, temp_array)

In [20]:
#plt.plot(år_array, temp_array, '.')
#plt.xlim(0,25)

In [21]:
GRIP_age = GRIPår0[9:-24]
NGRIP_age = NGRIPår0[16:-24]
GRIP_acc_rate = GRIPakk0[9:-24]
NGRIP_acc_rate = NGRIPakk0[16:-24]

#plt.plot(GRIP_age, GRIP_acc_rate)
#plt.plot(NGRIP_age, NGRIP_acc_rate)


gns_acc_rate1 = (GRIP_acc_rate + NGRIP_acc_rate)/2 #m is/år fra år 1980
gns_acc_rate = np.repeat(gns_acc_rate1, 365.25*4)*1/(365.25*4)

#print(gns_acc_rate)
#print(gns_acc_rate1/(365.25*4))

# Simulering

In [22]:
def langtidskørsel(k_acc, k_melt):    
    V = 2.99 * 10**15        #m^3
    H = 3 * 10**3            #m
    R = (3/16*V/H)**(1/2)          #m
    T_f = 0                  #grader
    alfa = - 9.8 * 10**(-3)  #temperaturændring pr. højdeforøgelse (deg C/m)
    k = H/np.sqrt(R)         #formfaktor. Vi antager, at iskappens form ikke ændrer sig
    dt = 1/(4*365.25)        #tidsskridt i år

    #k_melt = 8.5 * 5 * 2 * 10**(-1)  #Smeltekonstant    

    V_list = []
    V_melt_list = []         #For overbliks skyld
    acc_list = []
    R_list = []
    år_list = []

    N = 17000000
    tæller = 0

    for i, j in np.nditer([temp_array[0:N], gns_acc_rate[0:N]]):
    
        R = (3/16 * V/k)**(2/5)  #Jeppes formel. Undersøges senere
        V_acc = k_acc * (j * 8 * R**2)*dt  #Volumen tilført pr tidsskridt.

        if i - T_f < 0:
            V_melt = 0

        elif i == 0:
            V_melt = 0

        elif i - T_f > 0:
            V_melt = dt*k_melt * 8/3 * R * (i - T_f) * ((T_f - i)/(alfa * k))**2 #Vi undersøger dette integrale senere #Tiføj ekstra getjerkonstant fra isbjergedannelse. + k_isbjerg
            #og r_f = R - ((T_f - temp[t])/(alfa * k))**2

        V +=  - V_acc + V_melt  #Omvendt fortegn for at gå tilbage i tid

        if tæller % 500 == 0:
            V_list.append(V)
            V_melt_list.append(V_melt)
            acc_list.append(V_acc)
            R_list.append(R)
            år_list.append(år_array[tæller])
            tæller += 1
        else:
            tæller += 1
        
    V_list = np.array(V_list)
    R_list = np.array(R_list)
    år_list = np.array(år_list)

    return V_list, R_list, år_list

In [25]:
for i in tqdm(range(len(k_acc_værdier))):
    V_list, R_list, år_list = langtidskørsel(k_acc_værdier[i], k_melt_list[i])
    np.savez_compressed(f'Voluminer_holocæn{i}.npz', V_list = V_list)
    np.savez_compressed(f'Radier_holocæn{i}.npz', R_list = R_list)
    np.savez_compressed(f'årstal_holocæn{i}.npz', år_list = år_list)

np.savez_compressed('k_værdier_holocæn.npz', k_acc_værdier = k_acc_værdier)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [4:39:05<00:00, 167.46s/it] 


In [26]:
len(k_acc_værdier)

100

In [80]:
år = np.load("årstal_lang_test0.npz")
#åår = år['V_kørsler']
voluminer = np.load('Voluminer_lang_test0.npz')
år = år['år_list']
voluminer = voluminer['V_list']